In [1]:
import sys
import os
import json
from pathlib import Path
import tensorflow as tf
from keras import backend as K

In [2]:
base_path = '/Users/lorenzofederici/Univpm/Tesi/LUS-multitask-learning'
if base_path not in sys.path:
    sys.path.append(base_path)

# import custom lib
from src.utils.dataset import *
from bck.experiment import Experiment

gpu = len(tf.config.list_physical_devices('GPU'))>0
if gpu:
    tf.config.set_visible_devices([tf.config.list_physical_devices('GPU')[0],tf.config.list_physical_devices('CPU')[0]])
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
    print("---> GPU is available <---")

ModuleNotFoundError: No module named 'utils'

In [ ]:
exps_path = '/Users/lorenzofederici/Univpm/Tesi/LUS-multitask-learning/config/params.json'
idx = 2

with open(exps_path) as f:
    configs = json.load(f)

configs_general = configs['SETTING']
base_path       = configs_general['BASE_PATH']
configs_exps    = configs['EXPS'][idx]

print('>>Setting: ', configs_general)
print('>>Experiment: ', configs_exps)

In [ ]:
# K.clear_session
for config in configs_exps:
    experiment = Experiment(configs_general, config, output_mode= (1,0))

experiment.build_experiment()
experiment.split_dataset()
# model = experiment.build_model()
# model = experiment.compile_model(model)

# model.summary()

In [ ]:
experiment.generate_split_charts('ldistr')

In [ ]:
pkl_centersdict_path = os.path.join(
    experiment.base_path, 'data/iclus', 'hospitals-patients-dict.pkl'
)
medical_center_patients, data_index, data_map_idxs_pcm, score_counts, labels = load_dsdata_pickle(experiment.dataset, pkl_centersdict_path)

In [ ]:
score_counts

In [ ]:
import numpy as np
dizionario_originale = data_map_idxs_pcm
score = labels

nuovo_dizionario = {}

for chiave, lista_indici in dizionario_originale.items():
    lista_score = [score[i] for i in lista_indici]
    mean_score = int(np.mean(lista_score))
    nuovi_valori = [lista_indici, lista_score, mean_score]

    nuovo_dizionario[chiave] = nuovi_valori

print(nuovo_dizionario)


In [ ]:
indici = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
classi = [0, 1, 2, 3, 2, 1, 0, 3, 2, 1, 0, 3, 2, 1, 0, 3]
print("Dimensione di indici:", len(indici))
print("Dimensione di classi:", len(classi))


# Creazione del dizionario
class_movies = {}

# Iterazione sugli elementi degli array
for indice, classe in zip(indici, classi):
    # Verifica se la classe è già una chiave nel class_movies
    if classe in class_movies:
        # Se sì, aggiungi l'indice alla lista esistente
        class_movies[classe].append(indice)
    else:
        # Se no, crea una nuova chiave e associa l'indice come primo elemento della lista
        class_movies[classe] = [indice]

# Stampare il risultato
print(class_movies)


In [ ]:
import copy
def select_movie(rnd_cls, class_dict):
    """
    :param rnd_cls: one of the classes a pixel can be assigned to
    For a given class rnd_cls, determines the probability of sampling a certain movie based on the amount of available frames and returns a movie_id
    """
    
    # Find the amount of frames per movie in this class
    nr_frames_per_movie = []
    [nr_frames_per_movie.append(len(class_dict[rnd_cls][i])) for i in class_dict[rnd_cls]]

    # Bound the amount of frames on 50, to not explode probabilities for long movies
    nr_frames_per_movie = np.array(nr_frames_per_movie)
    nr_frames_per_movie_bounded = copy.deepcopy(nr_frames_per_movie)
    nr_frames_per_movie_bounded[nr_frames_per_movie>50] = 50

    # Compute the probabilities
    probs = 0.2*(nr_frames_per_movie_bounded-1)+1

    # Return the sampled movie, sampled using the defined probability distribution
    movie_index = np.argmax(np.random.multinomial(1, probs/np.sum(probs)),-1)
    return movie_index

def class_balanced_batch_generator(classes, batch_size, class_movies, class_dict, full_class_dict):
    """
    :param batch_size: mini-batch size used during training
    :param class_movies:  List containing per class a list of available movie names
    """
    x = []
    y = []

    no_shuffle = True

    if not no_shuffle:
        random.shuffle(classes)

    # Fill the batch with frames from different movies
    # If all movies contributes one frame and we extracted still fewer frames than batch_size, just start again
    for i in range(batch_size):

        # Determine class of next element in batch from precreated list
        rnd_cls = classes[i]

        # Select a movie from this class, with a probability that is related to the amount of frames that are still available
        rnd_movie = select_movie(rnd_cls, class_dict)

        print('set rnd movie: ', rnd_movie)
        # rnd_movie_name = class_movies[rnd_cls][rnd_movie]

        # nr_frames = len(class_dict[rnd_cls][rnd_movie_name])
        # if nr_frames == 0:
        #     class_dict[rnd_cls][rnd_movie_name] = copy.deepcopy(full_class_dict[rnd_cls][rnd_movie_name])
        #     #print(f'Fill again {rnd_movie_name} in class {rnd_cls}')
        #     nr_frames = len(class_dict[rnd_cls][rnd_movie_name])

        # rnd_frame = random.randrange(0,nr_frames)
        # print('set rnd frame: ' , rnd_frame)

        # frame = class_dict[rnd_cls][rnd_movie_name].pop(rnd_frame)

        # hf = h5py.File(Path(rnd_movie_name)/(frame+".hdf5"), 'r')

        # x.append(np.array(hf['image'], dtype=np.float32))
        # y.append(np.array(hf[self.dict_key], dtype=np.float32))

        # hf.close()

    # # Stack all elements from the batch in an array
    # x = np.stack(x)
    # y = np.stack(y)
    
    # return x,y

In [ ]:
class_dict = {
    0: {'movie1': 1, 'movie2': 2, 'movie1': 3, 'movie2': 4, 'movie1': 5, 'movie2': 6},
    1: {'movie3': 7, 'movie4': 8, 'movie3': 9, 'movie4': 10, 'movie3': 11, 'movie4': 112},
    2: {'movie5': 5, 'movie6': 6},
    3: {'movie7': 7, 'movie8': 8}
}
class_movies = []
batch_size = 8
classes = [0,0,0,1,2,3]*int(np.ceil((batch_size/6)))
full_class_dict = class_dict

for cl in range(4):
    class_movies.append(list(class_dict[cl].keys()))

# while True:
class_balanced_batch_generator(classes, batch_size, class_movies, class_dict, full_class_dict)
# final_x, y_onehot = self.process_xy(x,y)
# yield (final_x,y_onehot)